In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlrtugas/02-14-2018.csv


In [8]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/ids-intrusion-csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install sparkmagic
!pip install pyspark
#!ls ../input/ids-intrusion-csv -a

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col, isnan, when, trim
from pyspark.sql import DataFrame

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.classification import SVMWithSGD, SVMModel

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import RegressionEvaluator
from functools import reduce

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
sqlContext = SQLContext(spark.sparkContext)
data1 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-14-2018.csv')
data2 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-15-2018.csv')
data3 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-16-2018.csv')
data4 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-21-2018.csv')
data5 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-22-2018.csv')
data6 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-23-2018.csv')
data7 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-28-2018.csv')
data8 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/03-01-2018.csv')
data9 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/03-02-2018.csv')
data10 = sqlContext.read.format( 'com.databricks.spark.csv' ).options( header='true',inferSchema='true' ).csv( '/kaggle/input/ids-intrusion-csv/02-20-2018.csv')

data10 = data10.drop('Flow ID')
data10 = data10.drop('Src IP')
data10 = data10.drop('Src Port')
data10 = data10.drop('Dst IP')

def unionAll(*dfs):
    return reduce(DataFrame.unionAll,dfs)
def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))

data = unionAll(data1,data2,data3,data4,data5,data6,data7,data8,data9,data10)
#data = unionAll(data1)
data = data.drop('Timestamp')
data = data.filter(data.Label != 'Label')
#print(data.count())
data = data.select([to_null(c).alias(c) for c in data.columns]).na.drop()
#print(data.count())
#data.printSchema()


In [ ]:
#data.printSchema()
#data.select("Label").show()
#distinct_ids = [x.Label for x in data.select('Label').distinct().collect()]
#distinct_ids
counts = data.groupBy('Label').count().sort('count',ascending=False)
counts.show()

In [ ]:
stages = []
indexer = StringIndexer(inputCol='Label',outputCol='label').fit(data)
stages += [indexer]
#df_ind = indexer.transform(data)
#df_ind.printSchema()
#df = df_ind.drop('Label')
#df.printSchema()
#counts = df.groupBy('Label_index').count().sort('Label_index',ascending=False)
#counts.show()

In [ ]:
numeric_features = [t[0] for t in data.dtypes if t[1] != 'str']
#numeric_features
#df.select(numeric_features).describe().toPandas().transpose()

In [ ]:
numeric_features.pop()
numeric_features
assembler = VectorAssembler(inputCols=numeric_features,outputCol="features")
stages += [assembler]

In [ ]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(data)
df = pipelineModel.transform(data)
selectedCols = ['label','features'] + numeric_features
df = df.select(selectedCols)
#df.printSchema()

In [ ]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaled_df = standardScaler.fit(df).transform(df)

In [ ]:
#print(df.count())
(train, test) = scaled_df.randomSplit([0.8,0.2], seed=2020)
#train.columns

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=3, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(train)
#predictions = lrModel.transform(test)

In [ ]:
predictions = lrModel.transform(test)
#print(predictions.prediction)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="label")
print("F1 Score                        :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
print("Accuracy                        :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

print("Weighted Precision              :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})))
print("Weighted Recall                 :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})))

print("Weighted True Positive Rate     :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedTruePositiveRate"})))
print("Weighted False Positive Rate    :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedFalsePositiveRate"})))
print("Weighted F Measure              :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedFMeasure"})))

print("True Positive Rate by Label     :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "truePositiveRateByLabel"})))
print("False Positive Rate by Label    :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "falsePositiveRateByLabel"})))

print("Precision By Label              :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "precisionByLabel"})))
print("Recall By Label                 :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "recallByLabel"})))
print("F Measure by Label              :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "fMeasureByLabel"})))

print("Log Loss                        :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "logLoss"})))
print("Hamming Loss                    :" + str(evaluator.evaluate(predictions, {evaluator.metricName: "hammingLoss"})))
